### train.csv & test.csv
- **price_doc**: sale price (this is the target variable)
- **id**: transaction id
- **timestamp**: date of transaction
- **full_sq**: total area in square meters, including loggias, balconies and other non-residential areas
- **life_sq**: living area in square meters, excluding loggias, balconies and other non-residential areas
- **floor**: for apartments, floor of the building
- **max_floor**: number of floors in the building
- **material**: wall material
- **build_year**: year built
- **num_room**: number of living rooms
- **kitch_sq**: kitchen area
- **state**: apartment condition
- **product_type**: owner-occupier purchase or investment
- **sub_area**: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.
- **full_all**: subarea population
- **male_f**, **female_f**: subarea population by gender
- **young_***: population younger than working age
- **work_***: working-age population
- **ekder_***: retirement-age population
- **n_m_{all|male|female}**: population between n and m years old
- **build_count_***: buildings in the subarea by construction type or year
- **x_count_500**: the number of x within 500m of the property
- **x_part_500**: the share of x within 500m of the property
- **\_sqm_**: square meters
- **cafe_count_d_price_p**: number of cafes within d meters of the property that have an average bill under p RUB
- **trc_**: shopping malls
- **prom_**: industrial zones
- **green_**: green zones
- **metro_**: subway
- **\_avto_**: distances by car
- **mkad_**: Moscow Circle Auto Road
- **ttk_**: Third Transport Ring
- **sadovoe_**: Garden Ring
- **bulvar_ring_**: Boulevard Ring
- **kremlin_**: City center
- **zd_vokzaly_**: Train station
- **oil_chemistry_**: Dirty industry
- **ts_**: Power plant

### macro.csv
A set of macroeconomic indicators, one for each date.

- **timestamp**: Transaction timestamp
- **oil_urals**: Crude Oil Urals ($/bbl)
- **gdp_quart**: GDP
- **gdp_quart_growth**: Real GDP growth
- **cpi**: Inflation - Consumer Price Index Growth
- **ppi**: Inflation - Producer Price index Growth
- **gdp_deflator**: Inflation - GDP deflator
- **balance_trade**: Trade surplus
- **balance_trade_growth**: Trade balance (as a percentage of previous year)
- **usdrub**: Ruble/USD exchange rate
- **eurrub**: Ruble/EUR exchange rate
- **brent**: London Brent ($/bbl)
- **net_capital_export**: Net import / export of capital
- **gdp_annual**: GDP at current prices
- **gdp_annual_growth**: GDP growth (in real terms)
- **average_provision_of_build_contract**: Provision by orders in Russia (for the developer)
- **average_provision_of_build_contract_moscow**: Provision by orders in Moscow (for the developer)
- **rts**: Index RTS / return
- **micex**: MICEX index / return
- **micex_rgbi_tr**: MICEX index for government bonds (MICEX RGBI TR) / yield
- **micex_cbi_tr**: MICEX Index corporate bonds (MICEX CBI TR) / yield
- **deposits_value**: Volume of household deposits
- **deposits_growth**: Volume growth of population's deposits
- **deposits_rate**: Average interest rate on deposits
- **mortgage_value**: Volume of mortgage loans
- **mortgage_growth**: Growth of mortgage lending
- **mortgage_rate**: Weighted average rate of mortgage loans
- **grp**: GRP of the subject of Russian Federation where Apartment is located
- **grp_growth**: Growth of gross regional product of the subject of the Russian Federation where Apartment is located
- **income_per_cap**: Average income per capita 
- **real_dispos_income_per_cap_growth**: Growth in real disposable income of Population
- **salary**: Average monthly salary
- **salary_growth**: Growth of nominal wages
- **fixed_basket**: Cost of a fixed basket of consumer goods and services for inter-regional comparisons of purchasing power
- **retail_trade_turnover**: Retail trade turnover
- **retail_trade_turnover_per_cap**: Retail trade turnover per capita
- **retail_trade_turnover_growth**: Retail turnover (in comparable prices in% to corresponding period of previous year)
- **labor_force**: Size of labor force
- **unemployment**: Unemployment rate
- **employment**: Employment rate
- **invest_fixed_capital_per_cap**: Investments in fixed capital per capita
- **invest_fixed_assets**: Absolute volume of investments in fixed assets
- **profitable_enterpr_share**: Share of profitable enterprises
- **unprofitable_enterpr_share**: The share of unprofitable enterprises
- **share_own_revenues**: The share of own revenues in the total consolidated budget revenues
- **overdue_wages_per_cap**: Overdue wages per person
- **fin_res_per_cap**: The financial results of companies per capita
- **marriages_per_1000_cap**: Number of marriages per 1,000 people
- **divorce_rate**: The divorce rate / growth rate
- **construction_value**: Volume of construction work performed (million rubles)
- **invest_fixed_assets_phys**: The index of physical volume of investment in fixed assets (in comparable prices in% to the corresponding month of Previous year)
- **pop_natural_increase**: Rate of natural increase / decrease in Population (1,000 persons)
- **pop_migration**: Migration increase (decrease) of population
- **pop_total_inc**: Total population growth
- **childbirth**: Childbirth
- **mortality**: Mortality
- **housing_fund_sqm**: Housing Fund (sqm)
- **lodging_sqm_per_cap**: Lodging (sqm / pax)
- **water_pipes_share**: Plumbing availability (pax)
- **baths_share**: Bath availability (pax)
- **sewerage_share**: Canalization availability
- **gas_share**: Gas (mains, liquefied) availability
- **hot_water_share**: Hot water availability
- **electric_stove_share**: Electric heating for the floor
- **heating_share**: Heating availability
- **old_house_share**: Proportion of old and dilapidated housing, percent
- **average_life_exp**: Average life expectancy
- **infant_mortarity_per_1000_cap**: Infant mortality rate (per 1,000 children aged up to one year)
- **perinatal_mort_per_1000_cap**: Perinatal mortality rate (per 1,000 live births)
- **incidence_population**: Overall incidence of the total population
- **rent_price_4+room_bus**: rent price for 4-room apartment, business class
- **rent_price_3room_bus**: rent price for 3-room apartment, business class
- **rent_price_2room_bus**: rent price for 2-room apartment, business class
- **rent_price_1room_bus**: rent price for 1-room apartment, business class
- **rent_price_3room_eco**: rent price for 3-room apartment, econom class
- **rent_price_2room_eco**: rent price for 2-room apartment, econom class
- **rent_price_1room_eco**: rent price for 1-room apartment, econom class
- **load_of_teachers_preschool_per_teacher**: Load of teachers of preschool educational institutions (number of children per 100 teachers);
- **child_on_acc_pre_school**: Number of children waiting for the determination to pre-school educational institutions, for capacity of 100
- **load_of_teachers_school_per_teacher**: Load on teachers in high school (number of pupils in hugh school for 100 teachers)
- **students_state_oneshift**: Proportion of pupils in high schools with one shift, of the total number of pupils in high schools
- **modern_education_share**: Share of state (municipal) educational organizations, corresponding to modern requirements of education in the total number of high schools;
- **old_education_build_share**: The share of state (municipal) educational organizations, buildings are in disrepair and in need of major repairs of the total number.
- **provision_doctors**: Provision (relative number) of medical doctors in area
- **provision_nurse**: Provision of nursing staff
- **load_on_doctors**: The load on doctors (number of visits per physician)
- **power_clinics**: Capacity of outpatient clinics
- **hospital_beds_available_per_cap**: Availability of hospital beds per 100 000 persons
- **hospital_bed_occupancy_per_year**: Average occupancy rate of the hospital beds during a year
- **provision_retail_space_sqm**: Retail space
- **provision_retail_space_modern_sqm**: Provision of population with retail space of modern formats, square meter
- **retail_trade_turnover_per_cap**: Retail trade turnover per capita
- **turnover_catering_per_cap**: Turnover of catering industry per person
- **theaters_viewers_per_1000_cap**: Number of theaters viewers per 1000 population
- **seats_theather_rfmin_per_100000_cap**: Total number of seats in Auditorium of the Ministry of Culture Russian theaters per 100,000 population
- **museum_visitis_per_100_cap**: Number of visits to museums per 1000 of population
- **bandwidth_sports**: Capacity of sports facilities

### Libraries

In [ ]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as grdspc
import seaborn as sns
import random as rnd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor as RFR, GradientBoostingRegressor as GBR
from sklearn.model_selection import GridSearchCV as GSCV, cross_val_score as CVS

# 1. Baseline Model Development
## a. Baseline Model (from Task 2)

### Getting data from train.csv & test.csv
##### For original model from task 2

In [ ]:
SbrBkTrain = pd.read_csv('train.csv', parse_dates=['timestamp']).filter(['full_sq', 'life_sq', 'floor', 'max_floor', 'material', 'build_year', 'num_room', 'kitch_sq', 'state', 'product_type', 'sub_area', 'price_doc', 'timestamp'])
SbrBkTest = pd.read_csv('test.csv', parse_dates=['timestamp']).filter(['id', 'full_sq', 'life_sq', 'floor', 'max_floor', 'material', 'build_year', 'num_room', 'kitch_sq', 'state', 'product_type', 'sub_area', 'timestamp'])

### Original data cleaning

In [ ]:
# Train data cleaning
# full_sq should be a value between 7 and 500 from my observation
SbrBkTrain.full_sq.loc[SbrBkTrain.full_sq < 7] = np.nan
SbrBkTrain.full_sq.loc[SbrBkTrain.full_sq > 500] = np.nan
# life_sq should be a value between 7 and 300 from my observation
SbrBkTrain.life_sq.loc[SbrBkTrain.life_sq < 7] = np.nan
SbrBkTrain.life_sq.loc[SbrBkTrain.life_sq > 300] = np.nan
# floor should not be a value above 40 from my observation
SbrBkTrain.floor.loc[SbrBkTrain.floor > 40] = np.nan
# max_floor should not be a value above 50 from my observation
SbrBkTrain.max_floor.loc[SbrBkTrain.max_floor > 50] = np.nan
# max_floor should not be a value below the value of floor in the same row
SbrBkTrain.max_floor.loc[SbrBkTrain.max_floor < SbrBkTrain.floor] = np.nan
# build_year should not be before 1800 and above 2020 from my observation
SbrBkTrain.build_year.loc[SbrBkTrain.build_year < 1800] = np.nan 
SbrBkTrain.build_year.loc[SbrBkTrain.build_year > 2020] = np.nan
# num_room should be a value between 1 and 12 from my observation
SbrBkTrain.num_room.loc[SbrBkTrain.num_room < 1] = np.nan
SbrBkTrain.num_room.loc[SbrBkTrain.num_room > 12] = np.nan
# kitch_sq should not be a value below 2 and above 250 from my observation
SbrBkTrain.kitch_sq.loc[SbrBkTrain.kitch_sq < 2] = np.nan
SbrBkTrain.kitch_sq.loc[SbrBkTrain.kitch_sq > 250] = np.nan
# full_sq should not be a value below life_sq in the same row
SbrBkTrain.life_sq.loc[SbrBkTrain.life_sq > SbrBkTrain.full_sq] = np.nan
# full_sq should not be a value below kitch_sq in the same row
SbrBkTrain.kitch_sq.loc[SbrBkTrain.full_sq < SbrBkTrain.kitch_sq] = np.nan
# kitch_sq part of full_sq should not be above life_sq part of full_sq in the same row, as long as full_sq is greater than the sum of life_sq and kitch_sq
SbrBkTrain.kitch_sq.loc[(SbrBkTrain.kitch_sq / SbrBkTrain.full_sq > SbrBkTrain.life_sq / SbrBkTrain.full_sq) & (SbrBkTrain.full_sq >= SbrBkTrain.life_sq + SbrBkTrain.kitch_sq)] = np.nan
# full_sq should not be a value below the sum of life_sq and kitch_sq in the same row
SbrBkTrain.life_sq.loc[SbrBkTrain.full_sq < SbrBkTrain.life_sq + SbrBkTrain.kitch_sq] = SbrBkTrain.full_sq[SbrBkTrain.full_sq < SbrBkTrain.life_sq + SbrBkTrain.kitch_sq] - SbrBkTrain.kitch_sq.loc[SbrBkTrain.full_sq < SbrBkTrain.life_sq + SbrBkTrain.kitch_sq]
# fixing the state values that are incorrect due to misstyping
SbrBkTrain.state.loc[SbrBkTrain.state > 4] = SbrBkTrain.state.loc[SbrBkTrain.state > 4] // 10

# Test data cleaning
# full_sq should be a value between 7 and 500 from my observation
SbrBkTest.full_sq.loc[SbrBkTest.full_sq < 7] = np.nan
SbrBkTest.full_sq.loc[SbrBkTest.full_sq > 500] = np.nan
# life_sq should be a value between 7 and 300 from my observation
SbrBkTest.life_sq.loc[SbrBkTest.life_sq < 7] = np.nan
SbrBkTest.life_sq.loc[SbrBkTest.life_sq > 300] = np.nan
# floor should not be a value above 40 from my observation
SbrBkTest.floor.loc[SbrBkTest.floor > 40] = np.nan
# max_floor should not be a value above 50 from my observation
SbrBkTest.max_floor.loc[SbrBkTest.max_floor > 50] = np.nan
# max_floor should not be a value below the value of floor in the same row
SbrBkTest.max_floor.loc[SbrBkTest.max_floor < SbrBkTest.floor] = np.nan
# build_year should not be before 1800 and above 2020 from my observation
SbrBkTest.build_year.loc[SbrBkTest.build_year < 1800] = np.nan 
SbrBkTest.build_year.loc[SbrBkTest.build_year > 2020] = np.nan
# num_room should be a value between 1 and 12 from my observation
SbrBkTest.num_room.loc[SbrBkTest.num_room < 1] = np.nan
SbrBkTest.num_room.loc[SbrBkTest.num_room > 12] = np.nan
# kitch_sq should not be a value below 2 and above 250 from my observation
# SbrBkTest.kitch_sq[SbrBkTest.kitch_sq < 2] = np.nan
# SbrBkTest.kitch_sq[SbrBkTest.kitch_sq > 250] = np.nan
# full_sq should not be a value below life_sq in the same row
SbrBkTest.life_sq.loc[SbrBkTest.life_sq > SbrBkTest.full_sq] = np.nan
# full_sq should not be a value below kitch_sq in the same row
SbrBkTest.kitch_sq.loc[SbrBkTest.full_sq < SbrBkTest.kitch_sq] = np.nan
# kitch_sq part of full_sq should not be above life_sq part of full_sq in the same row, as long as full_sq is greater than the sum of life_sq and kitch_sq
SbrBkTest.kitch_sq.loc[(SbrBkTest.kitch_sq / SbrBkTest.full_sq > SbrBkTest.life_sq / SbrBkTest.full_sq) & (SbrBkTest.full_sq >= SbrBkTest.life_sq + SbrBkTest.kitch_sq)] = np.nan
# full_sq should not be a value below the sum of life_sq and kitch_sq in the same row
SbrBkTest.life_sq.loc[SbrBkTest.full_sq < SbrBkTest.life_sq + SbrBkTest.kitch_sq] = SbrBkTest.full_sq.loc[SbrBkTest.full_sq < SbrBkTest.life_sq + SbrBkTest.kitch_sq] - SbrBkTest.kitch_sq[SbrBkTest.full_sq < SbrBkTest.life_sq + SbrBkTest.kitch_sq]
# fixing the state values that are incorrect due to misstyping
SbrBkTest.state.loc[SbrBkTest.state > 4] = SbrBkTest.state.loc[SbrBkTest.state > 4] // 10

### Original Handling of missing values

##### Viewing abnormalities

In [ ]:
print(f'Precentages of missing values in Train:\n{(SbrBkTrain.isna().sum() / SbrBkTrain.shape[0] * 100).round(2).sort_values(ascending = False)}')

##### handling abnormalities

In [ ]:
SbrBkTrain.drop(columns = ['kitch_sq'], inplace = True)
SbrBkTest.drop(columns = ['kitch_sq'], inplace = True)

SbrBkTrain['toDrop'] = (((SbrBkTrain.build_year.isna()) | (SbrBkTrain.state.isna())) & (SbrBkTrain.isna().sum(axis = 1) > 5))
# marking rows that are randomly selected to be dropped
SbrBkTrain.toDrop[rnd.sample(list((((SbrBkTrain.build_year.isna()) | (SbrBkTrain.state.isna())) & (SbrBkTrain.isna().sum(axis = 1) == 5))[(((SbrBkTrain.build_year.isna()) | (SbrBkTrain.state.isna())) & (SbrBkTrain.isna().sum(axis = 1) == 5))==True].index), 3300)] = True
# dropping all marked rows
SbrBkTrain = SbrBkTrain[~SbrBkTrain.toDrop]
SbrBkTrain.drop(columns = ['toDrop'], inplace = True)

# viewing the precentages again
print(f'Precentages of missing values in Train:\n{(SbrBkTrain.isna().sum() / SbrBkTrain.shape[0] * 100).round(2).sort_values(ascending = False)}')
print(f'Precentages of missing values in Test:\n{(SbrBkTest.isna().sum() / SbrBkTest.shape[0] * 100).round(2).sort_values(ascending = False)}')

### Imputing missing values

In [ ]:
SbrBkTrain['product_typeCode'] = pd.Categorical(SbrBkTrain.product_type).codes
SbrBkTest['product_typeCode'] = pd.Categorical(SbrBkTest.product_type).codes
SbrBkTest.product_typeCode[SbrBkTest.product_typeCode == -1] = np.nan
SbrBkTrain['sub_areaCode'] = pd.Categorical(SbrBkTrain.sub_area).codes
SbrBkTest['sub_areaCode'] = pd.Categorical(SbrBkTest.sub_area).codes

imputedTrainData = KNNImputer(n_neighbors = 20).fit_transform(SbrBkTrain[['full_sq', 'life_sq', 'floor', 'max_floor', 'material', 'build_year', 'num_room', 'state', 'product_typeCode', 'sub_areaCode']])
imputedTestData = KNNImputer(n_neighbors = 20).fit_transform(SbrBkTest[['full_sq', 'life_sq', 'floor', 'max_floor', 'material', 'build_year', 'num_room', 'state', 'product_typeCode', 'sub_areaCode']])
imputedTrain = pd.DataFrame(imputedTrainData, columns = ['full_sq', 'life_sq', 'floor', 'max_floor', 'material', 'build_year', 'num_room', 'state', 'product_typeCode', 'sub_areaCode'])
imputedTest = pd.DataFrame(imputedTestData, columns = ['full_sq', 'life_sq', 'floor', 'max_floor', 'material', 'build_year', 'num_room', 'state', 'product_typeCode', 'sub_areaCode'])
# as observed previously all of the features are integers, and they should remain as such
imputedTrain = imputedTrain.round()
imputedTest = imputedTest.round()
imputedTrain = imputedTrain.set_index(SbrBkTrain.index)
imputedTest = imputedTest.set_index(SbrBkTest.index)
imputedTrain['product_type'] = SbrBkTrain.product_type
imputedTest['product_type'] = SbrBkTest.product_type
imputedTrain['sub_area'] = SbrBkTrain.sub_area
imputedTest['sub_area'] = SbrBkTest.sub_area
imputedTrain['price_doc'] = SbrBkTrain.price_doc
imputedTest['id'] = SbrBkTest.id
imputedTrain['timestamp'] = SbrBkTrain.timestamp
imputedTest['timestamp'] = SbrBkTest.timestamp
imputedTrain.product_type.loc[imputedTrain.product_type.isna()] = np.where(imputedTrain.product_typeCode[imputedTrain.product_type.isna()] == 0, 'Investment', 'OwnerOccupier')
imputedTest.product_type.loc[imputedTest.product_type.isna()] = np.where(imputedTest.product_typeCode[imputedTest.product_type.isna()] == 0, 'Investment', 'OwnerOccupier')
imputedTrain.drop(columns = ['product_typeCode', 'sub_areaCode'], inplace = True)
imputedTest.drop(columns = ['product_typeCode', 'sub_areaCode'], inplace = True)

### Original feature engineering

In [ ]:
imputedTrain = imputedTrain.assign(roomPerSq = imputedTrain.num_room / imputedTrain.full_sq,
                                   buildingAge = imputedTrain.timestamp.dt.year - imputedTrain.build_year,
                                   practAging = imputedTrain.buildingAge / imputedTrain.state,
                                   roomSize = imputedTrain.life_sq / imputedTrain.num_room,
                                   heightScore = (imputedTrain.floor - imputedTrain.max_floor * 0.37).abs() / (imputedTrain.max_floor * 0.63 / 10),
                                   lifeRatio = imputedTrain.life_sq / (imputedTrain.full_sq / 10),
                                   month = imputedTrain.timestamp.dt.month)
imputedTest = imputedTest.assign(roomPerSq = imputedTest.num_room / imputedTest.full_sq,
                                 buildingAge = imputedTest.timestamp.dt.year - imputedTest.build_year,
                                 practAging = imputedTest.buildingAge / imputedTest.state,
                                 roomSize = imputedTest.life_sq / imputedTest.num_room,
                                 heightScore = (imputedTest.floor - imputedTest.max_floor * 0.37).abs() / (imputedTest.max_floor * 0.63 / 10),
                                 lifeRatio = imputedTest.life_sq / (imputedTest.full_sq / 10),
                                 month = imputedTest.timestamp.dt.month)
imputedTrain.heightScore[imputedTrain.heightScore.isna()] = 7
imputedTest.heightScore[imputedTest.heightScore.isna()] = 7

### Twicked from Original Label encoding

In [ ]:
for col in SbrBkTrain.columns:
    if SbrBkTrain[col].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(SbrBkTrain[col].values) + list(SbrBkTest[col].values))
        SbrBkTrain[col] = lbl.transform(list(SbrBkTrain[col].values))
        SbrBkTest[col] = lbl.transform(list(SbrBkTest[col].values))

# addition none object labeling
mtrlLbl = LabelEncoder().fit(list(SbrBkTrain.material.values) + list(SbrBkTest.material.values))
stateLbl = LabelEncoder().fit(list(SbrBkTrain.state.values) + list(SbrBkTest.state.values))
SbrBkTrain.material = mtrlLbl.transform(SbrBkTrain.material.values)
SbrBkTest.material = mtrlLbl.transform(SbrBkTest.material.values)
SbrBkTrain.state = stateLbl.transform(SbrBkTrain.state.values)
SbrBkTest.state = stateLbl.transform(SbrBkTest.state.values)

### Original Rescaling

In [ ]:
# Rescaling full_sq in imputedTrain and impuedTest
scaledMaxFullSq = imputedUnion.full_sq.max() / 100
imputedTrain.full_sq = imputedTrain.full_sq / scaledMaxFullSq
imputedTest.full_sq = imputedTest.full_sq / scaledMaxFullSq

# Rescaling life_sq in imputedTrain and impuedTest
scaledMaxLifeSq = imputedUnion.life_sq.max() / 100
imputedTrain.life_sq = imputedTrain.life_sq / scaledMaxLifeSq
imputedTest.life_sq = imputedTest.life_sq / scaledMaxLifeSq

# Rescaling build_year in imputedTrain and impuedTest
minBuildYear = imputedUnion.build_year.min()
imputedTrain.build_year = imputedTrain.build_year - minBuildYear
imputedTest.build_year = imputedTest.build_year - minBuildYear

### Original Train/Validation/Test split

In [ ]:
finalTrain = imputedTrain.copy(deep = True).reset_index(drop = True)
finalTest = imputedTest.copy(deep = True).reset_index(drop = True)

# due to it being a time series it is only appropriate to split the data in a chronological manner
TrainData = finalTrain.loc[:int(finalTrain.shape[0] * 0.7), :]
xTrn = TrainData.drop(columns = ['price_doc', 'logPrice', 'timestamp'])
yTrn = TrainData.logPrice
ValidationData = finalTrain.loc[int(finalTrain.shape[0] * 0.7): int(finalTrain.shape[0] * 0.85), :]
xVal = ValidationData.drop(columns = ['price_doc', 'logPrice', 'timestamp'])
yVal = ValidationData.logPrice
TestData = finalTrain.loc[int(finalTrain.shape[0] * 0.85):, :]
xTst = TestData.drop(columns = ['price_doc', 'logPrice', 'timestamp'])
yTst = TestData.logPrice

### Original Random Forest Model

In [ ]:
ftrSlctModel = RFR(n_jobs = -1, random_state = 42)
ftrSlctModel.fit(xTrn, yTrn)
imprtnc = ftrSlctModel.feature_importances_
features = pd.Series(imprtnc, index = xTrn.columns).sort_values(ascending = False)
features = list(features[features > 0.05].index)

rndFrstBaseModel = RFR()
rndFrstParams = {
    'max_depth': [i for i in range(4, 9, 2)],
    'n_estimators': [i for i in range(100, 301, 50)],
    'min_samples_leaf': [i for i in range(3, 10, 3)]
}

grdSrcRndFrst = GSCV(rndFrstBaseModel, rndFrstParams, cv = 5, scoring = 'neg_mean_squared_error')
grdSrcRndFrst.fit(xTrn[features], yTrn)

RndFrstModelParams = grdSrcRndFrst.best_params_

rndFrstOptModel = RFR(**RndFrstModelParams)
rndFrstOptModel.fit(xTrn[features], yTrn)

rndFrstMseValScores = -1 * CVS(rndFrstOptModel, xVal[features], yVal, cv = 5, scoring = 'neg_mean_squared_error')
rndFrstMseTstScores = -1 * CVS(rndFrstOptModel, xTst[features], yTst, cv = 5, scoring = 'neg_mean_squared_error')

print(f'The mean squared error of the model on the validation set is {rndFrstMseValScores.mean()}')
print(f'The mean squared error of the model on the Test set is {rndFrstMseTstScores.mean()}')

### Original XGBoost Model

In [ ]:
xgbBaseModel = GBR()

xgbParams = {
    'learning_rate': [i/1000 for i in range(6, 15, 4)],
    'n_estimators': [i for i in range(100, 301, 100)],
    'subsample': [i/20 for i in range(13, 18, 2)],
    'max_depth': [i for i in range(4, 9, 2)],
    'min_samples_leaf': [i for i in range(3, 18, 7)]
}

grdSrcXgb = GSCV(xgbBaseModel, xgbParams, cv = 5, scoring = 'neg_mean_squared_error')
grdSrcXgb.fit(xTrn, yTrn)

xgbModelParams = grdSrcXgb.best_params_

xgbOptModel = GBR(**xgbModelParams)
xgbOptModel.fit(xTrn, yTrn)

xgbMseValScores = -1 * CVS(xgbOptModel, xVal, yVal, cv = 5, scoring = 'neg_mean_squared_error')
xgbMseTstScores = -1 * CVS(xgbOptModel, xTst, yTst, cv = 5, scoring = 'neg_mean_squared_error')

print(f'The mean squared error of the model on the validation set is {xgbMseValScores.mean()}')
print(f'The mean squared error of the model on the Test set is {xgbMseTstScores.mean()}')

### Original Submission Output

In [ ]:
# export random forest model
yPredRndFrst = np.round(np.exp(rndFrstOptModel.predict(finalTest[features])), decimals = -3)
rndFrstSubmission = pd.DataFrame({'id': finalTest.id, 'price_doc': yPredRndFrst})
rndFrstSubmission.to_csv('RandomForestSubmission.csv', index = False)

# export xgboost model
yPredXgb = np.round(np.exp(xgbOptModel.predict(finalTest[[col for col in finalTest.columns if col not in ['id', 'timestamp']]])), decimals = -3)
xgbSubmission = pd.DataFrame({'id': finalTest.id, 'price_doc': yPredXgb})
xgbSubmission.to_csv('XGBoostSubmission.csv', index = False)

## b. Baseline Results
need to discuss the model's performance metrics

therefor need to calculate the model's performance metrics

# 2. Notebook Analysis
## a. Notebook Selection
need to choose a notebook

Tom recomendation:
- combination of 'simple-exploration-notebook-sberbank.ipynb' and 'feature-engineering-validation-strategy.ipynb'
- 'basic-time-series-analysis-feature-selection.ipynb'
- 'sberbank-russian-housing-market.ipynb' (using PCA, very useful for us i think)
- 'a-very-extensive-exploratory-analysis-in-python.ipynb' (havent gone through all of it yet, but seems good in first look)
- 'a-very-extensive-sberbank-exploratory-analysis.Rmd' (havent gone through all of it yet, but seems good in first look)

## b. Analysis Summary
after choosing notebook(s), summarize its methodology, results and conclusions, 
###### maybe for further use in the project

## c. Critique and Improvements
give some form of a feedback on how good is the approach of the chosen notebook(s) and what can be improved in it, giving suggestions for improvements and alternatives.

# 3. Model Enhancement
## a. Feature Engineering
we need to perform some feature engineering in a more informed method and explain the rational behind the method and the features

maybe examine the correlation of variables and give out a list of low correlated features to make features out of and then use rational to create the features hinted within them, using heatmap or something

## b. Enhanced Model Development
Explain new found rational of the steps of task 2 as it being redone
- data cleaning
    - illogical values
    - missing values
    - highly correlated values (heatmap or something)
    - imputation
- feature engineering
    - using rational explained in previous Sub-Task
    - testing specific effectiveness of engineered features using the jointplot function examplified in 'feature-engineering-validation-strategy.ipynb'
- Labeling and Rescaling
- Spliting data
- Model Creation
    - Cross validation hyper parameters ranges
    - Base Function Parameters
    - performance testing

# 4. Advanced Modeling
## a. Model Implementation
Perform Sub-Tasks 3.a and 3.b in code
## b. Evaluation and Comparison
Compare the results of the models by viewing and discussing their performance metrics

### Getting data from train.csv, test.csv and macro.csv
##### For new model influenced by notebooks

In [ ]:
SbrBkTrain = pd.read_csv('train.csv', parse_dates=['timestamp']).filter(['full_sq', 'life_sq', 'floor', 'max_floor', 'material', 'build_year', 'num_room', 'kitch_sq', 'state', 'product_type', 'sub_area', 'price_doc', 'timestamp'])
SbrBkTest = pd.read_csv('test.csv', parse_dates=['timestamp']).filter(['id', 'full_sq', 'life_sq', 'floor', 'max_floor', 'material', 'build_year', 'num_room', 'kitch_sq', 'state', 'product_type', 'sub_area', 'timestamp'])
SbrBkTrain = pd.merge(SbrBkTrain, pd.read_csv('macro.csv', parse_dates=['timestamp']), how = 'left', on='timestamp')
SbrBkTest = pd.merge(SbrBkTest, pd.read_csv('macro.csv', parse_dates=['timestamp']), how = 'left', on='timestamp')

## Need to make
### Handling illogical values

## Need to Make
### Handling missing values
- too much missing values in columns
- impute the rest

##### missing values
- normalize and impute them? (done in 'basic-time-series-analysis-feature-selection.ipynb')

### for data cleaning
##### Handling Outliers

In [ ]:
# in price_doc
ulimit = np.percentile(SbrBkTrain.price_doc.values, 99)
llimit = np.percentile(SbrBkTrain.price_doc.values, 1)
SbrBkTrain['price_doc'].ix[SbrBkTrain.price_doc>ulimit] = ulimit
SbrBkTrain['price_doc'].ix[SbrBkTrain.price_doc<llimit] = llimit

# in others, if any

### potential features to make

#### timestamp features:
- yearNmonth
- yearNweek
- year
- monthOyear
- weekOyear
- dayOyear
- yearNmonthCount- number of houses available in yearNmonth time period: (yearNmonth[i] == yearNmonth).sum() (maybe implement using a loop)
- yearNweekCount- number of houses available in yearNweek time period: (yearNweek[i] == yearNweek).sum() (maybe implement using a loop)
- lastNdaysCount- number of houses sold in the last n days: using a transformed timestamp to be an int value of the day which is called dayInt, ((dayInt[i] - dayInt <= n) & (dayInt[i] - dayInt >= 0)).sum()

#### ratio features:
- lifeSqPRfullSq: life_sq[i] / max(full_sq, 1)
- lifeSqRfullSq: life_sq[i] / full_sq[i]
- kitchSqPRlifeSq: kitch_sq[i] / max(life_sq, 1)
- kitchSqRlifeSq: kitch_sq[i] / life_sq[i]
- kitchSqPRfullSq: kitch_sq[i] / max(full_sq, 1)
- kitchSqRfullSq: kitch_sq[i] / full_sq[i]
- floorRmaxFloor: floor[i] / max_floor[i]
- preschoolRatio: children_preschool[i] / preschool_quota[i]
- schoolRatio: children_school[i] / school_quota[i]

#### other features:
- number of nulls in the row
- number of nulls in the row in accordance to weights related to the number of nulls in the collumn
- number of floors from top: max_floor[i] - floor[i]
- extraSq: full_sq[i] - life_sq[i] - kitch_sq[i]
- buildAge: build_year[i] - year[i]

### Label Encoding

In [ ]:
for col in SbrBkTrain.columns:
    if SbrBkTrain[col].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(SbrBkTrain[col].values) + list(SbrBkTest[col].values))
        SbrBkTrain[col] = lbl.transform(list(SbrBkTrain[col].values))
        SbrBkTest[col] = lbl.transform(list(SbrBkTest[col].values))

# addition none opject labeling
mtrlLbl = LabelEncoder().fit(list(SbrBkTrain.material.values) + list(SbrBkTest.material.values))
stateLbl = LabelEncoder().fit(list(SbrBkTrain.state.values) + list(SbrBkTest.state.values))
SbrBkTrain.material = mtrlLbl.transform(SbrBkTrain.material.values)
SbrBkTest.material = mtrlLbl.transform(SbrBkTest.material.values)
SbrBkTrain.state = stateLbl.transform(SbrBkTrain.state.values)
SbrBkTest.state = stateLbl.transform(SbrBkTest.state.values)

### Features Rescaling

##### feature selection:
- select features by looking at correlation matrix
    - out of all variables that are highly correlated with the log(price_doc) choose the most correlated features out of each highly correlated group
    - maybe vice versa?
    - keep only relevant features
    - using feature importance aswell
- take method from 'basic-time-series-analysis-feature-selection.ipynb'

### Multiple types of modeling and cross validating

### Submission Outputs

# 5. State-Of-The-Art Techniques
## a. Technique Exploration
further inspection of notebooks from the kaggle competition of the top competitors, searching for advanced techniques to apply in our models to improve them
## b. Application and Performance
discussing and explaining the works of these techniques, how and why they work.

###### maybe even implement them within our project.

# Presentation
#### Details
5-minute powerpoint presentation that describes the most interesting turning points in the project

i think we should include the saving of rows when data cleaning.

# Grading Criteria
- **Clarity and organization:**

    the report (presentation and PDF) should have structure and clear in the writing to make it easier for the readers to understand the process and findings
- **Comprehensiveness:**

    the report (presentation and PDF) should cover all aspects of the tasks having detailed explanations of our approach, analysis and conclusions
- **Innovation:**

    Creativity and originality is encouraged
- **Accuracy and Performance:**

    we need to show the improvement of the models when implementing the methodologies and compare the original results with the new ones
- **Relative Performance Evaluation:**

    the final grade is also assigned based on the performance in the course (sounds like bullshit because they dont have a way to check our performance in the course if attendance is was not a must)